In [15]:
import glob
import numpy as np
import pandas as pd
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import CuDNNLSTM, LSTM, Bidirectional
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, History, Callback
import matplotlib.pyplot as plt
import time
import os
import sys

from collections import Counter

# From stylenet testings "check piano representation"
import pretty_midi
import matplotlib.pyplot as plt
import os
import lib.file_util_v2
from lib.midi_util_v2 import midi_to_array_one_hot, stylify
from mido import MidiFile
from random import shuffle

import pickle

#from lib.midi_util import 


# The MIDI pitches we use.
# PITCHES = range(21,109,1)
# OFFSET = 109-21

# # The reverse of what is in encoding
# PITCHES_MAP = { i : p for i, p in enumerate(PITCHES) }
# print(len(PITCHES))




# A state is composed of the order of pitches contained in PITCHES_MAP
# state = 3*length of pitches_map
# where each pitch_state in a state is composed of [active or not, event, velocity]
PITCHES_TO_INDEX = {}
INDEX_TO_PITCHES = {}

# STATES TO INDICES
STATES_TO_INDEX = {}
INDEX_TO_STATES = {}



"""
######################################################

Setting Input and Output Folder

######################################################
"""

#INPUT_FOLDER = './input/magenta_2013'
#OUTPUT_FOLDER = './output/magenta_2013'
#INPUT_FOLDER = './input/Bach_Fuga sopra il Magnificat in D minor'
#OUTPUT_FOLDER = './output/Bach_Fuga sopra il Magnificat in D minor'
# INPUT_FOLDER = 'input/Bach_wv1041a'
#INPUT_FOLDER = 'input/Golden_Sun_2'
#OUTPUT_FOLDER = 'output/Golden_Sun_2_v0'
# OUTPUT_FOLDER = 'output/Bach_wv1041a'

INPUT_FOLDER = './data/preprocessed/single_piano/midi_bts_boy_with_luv'
OUTPUT_FOLDER = './output/lstm_single_instrument_v2/midi_bts_boy_with_luv'

PREPROCESSED_FOLDER =  OUTPUT_FOLDER + '/preprocessed'


# INPUT_FOLDER = './input/midi_bts_single_boy_with_luv'
# OUTPUT_FOLDER = './output/midi_bts_single_boy_with_luv_input_100'

if not os.path.exists(INPUT_FOLDER):
    print("Input folder: {} does not exist".format(INPUT_FOLDER))
    sys.exit()

if not os.path.exists(OUTPUT_FOLDER):
    print("Creating folder: {}".format(OUTPUT_FOLDER))
    os.makedirs(OUTPUT_FOLDER)

"""
######################################################

Custom Training Callbacks

Custom callback for generating data as the model trains (so we can view the data immediately)

######################################################
"""
class ModelCheckpoint_GenerateData(Callback):
    """Generate data with model every given time period
    """

    def __init__(self, filepath, notes, network_input, period=1, verbose=True):
        super(ModelCheckpoint_GenerateData, self).__init__()
        self.filepath = filepath
        self.period = period
        self.epochs_since_last_save = 0
        self.verbose = verbose
        # Data needed for data generation
        self.notes = notes
        self.network_input = network_input

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epochs_since_last_save += 1
        if self.epochs_since_last_save >= self.period:
            self.epochs_since_last_save = 0
            filepath = self.filepath.format(epoch=epoch+1, **logs)
            # Do something with the model
            length_notes =  np.unique(self.notes, axis=0).shape[0]
            print('length_notes: ')
            print(length_notes)

            prediction_output, random_seed = generate_notes(self.model, self.notes, self.network_input, length_notes)
            #print('prediction_output')
            #print(prediction_output)

            create_midi(prediction_output, filepath, self.notes)
            if self.verbose > 0:
                 print('\nEpoch %05d: saving data to %s' % (epoch + 1, filepath))

"""
######################################################

Data Preprocessing

######################################################
"""


def get_notes_and_velocities():
    """
        Get all the notes from the midi files

        Creates the PITCHES_TO_INDEX and INDEX_TO_PITCHES mapping for the pitch dictionaries

        returns the list of 


    """
    global PITCHES_TO_INDEX
    global INDEX_TO_PITCHES


    notes = np.array([])
    velocities = np.array([])


    # CREATE PITCHES_MAP, a mapping of unique values
    # a dictionary always has unique values

    # first count the number of pitches for all files
    # for instrument in midi_data.instruments:
    #     print(instrument)
    #     for i, note in enumerate(instrument.notes):
    #         print(note)
    #         note_name = pretty_midi.note_number_to_name(note.pitch)
    #         print(note_name)
    #         print(pretty_midi.note_name_to_number(note_name))
    #         if (i > 5):
    #             break

    list_notes = []
    list_mid_data = []
    for file in glob.glob(INPUT_FOLDER + "/*.mid"):
        midi_data = pretty_midi.PrettyMIDI(file)
        for instrument in midi_data.instruments:
            print(instrument)
            for i, note in enumerate(instrument.notes):
                #print(note)
                #note_name = pretty_midi.note_number_to_name(note.pitch)
                list_notes.append(note.pitch)
                #list_notes.append(note_name)
                #print(pretty_midi.note_name_to_number(note_name))
    print(len(list_notes))
    set_notes = set(item for item in list_notes)
    num_unique_notes = len(set_notes)
    print(num_unique_notes)

    # Create the mappings
    for i, item in enumerate(set_notes):
        PITCHES_TO_INDEX[item] = i
        INDEX_TO_PITCHES[i] = item

    print(len(PITCHES_TO_INDEX))
    print(PITCHES_TO_INDEX)
    print(len(INDEX_TO_PITCHES))
    print(INDEX_TO_PITCHES)
    #import pickle

    file_path = OUTPUT_FOLDER + '/PITCHES_TO_INDEX.pkl'
    with open(file_path, "wb") as f:
        pickle.dump(PITCHES_TO_INDEX, f)

    file_path = OUTPUT_FOLDER + '/INDEX_TO_PITCHES.pkl'
    with open(file_path, "wb") as f:
        pickle.dump(INDEX_TO_PITCHES, f)

    list_midi_data = None
    # Get the quantized midi to array of each file
    for file in glob.glob(INPUT_FOLDER + "/*.mid"):

        #midi = converter.parse(file)

        print("Parsing %s" % file)
        #mid_path = os.path
        try:
            midi_data = pretty_midi.PrettyMIDI(file)
            mid = MidiFile(file)
        except (KeyError, IOError, IndexError, EOFError, ValueError):
            print("NAUGHTY")
            break

        midi_array = midi_to_array_one_hot(mid, 4, PITCHES_TO_INDEX)
        print("[MIDI_ARRAY]")
        print(midi_array)
        print(midi_array.shape)
        #test = midi_array.tolist()
        #print(test[0])
        #print(len(test[0]))
        #notes.append(notes, midi_array[:])
        #notes = np.concatenate((notes, midi_array), axis=0)
        #print(notes[0:5])
        #print(notes.shape)

        # Add in each song as part of a big sequence.
        # These are in effect the list of NON-UNIQUE sequnetial states
        if list_midi_data == None:
            list_midi_data = midi_array
        else:
            list_midi_data = np.vstack((list_midi_data, midi_array))
        #list_midi_data = np.concatenate((list_midi_data, midi_array), axis=0)
        #break

    return list_midi_data

def prepare_sequences(notes, n_vocab):
    """ 
        Prepare the sequences used by the Neural Network.
        Converts the sequences of states into embeddings based on list of unique states
        Returns
            network_input
            network_output


    """
    sequence_length = 100
    #print(len(notes))
    #print(notes.shape)
    #print(notes[0])

    #n = np.unique(notes, axis=0)
    #print(n[0])
    #print(n.shape)
    #pitchnames = n.tolist()
    #print(len(n))
    #print(len(pitchnames))
    #print(pitchnames[0:5])
    print(notes.shape)
    temp_notes = notes.tolist()
    print(temp_notes[0:5])
    # get all pitch names
    #pitchnames = sorted(set(item for item in notes))

    # create a dictionary to map pitches to integers
    #Counter(str(e) for e in li)

    #note_to_int = dict((str(note), number) for number, note in enumerate(pitchnames))
    #print("len(note_to_int)")
    #print(len(note_to_int))
    #print(note_to_int[0])
    #print(note_to_int)
    network_input = []
    network_output = []

    print(STATES_TO_INDEX)
    # create input sequences and the corresponding outputs
    for i in range(0, len(temp_notes) - sequence_length, 1):
        sequence_in = temp_notes[i:i + sequence_length]
        sequence_out = temp_notes[i + sequence_length]
        network_input.append([STATES_TO_INDEX[str(state)] for state in sequence_in])
        #print(network_input[i])
        #print(len(network_input))
        network_output.append(STATES_TO_INDEX[str(sequence_out)])
        #print(network_output[i])
        #print(len(network_output))

    #print(network_input[0])
    #print(network_output[0])

    # reshape the input into a format compatible with LSTM layers
    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    #print(network_input.shape)
    # normalize input between 0 and 1
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)
    #print(network_output.shape)
    #print(network_output[0:5])
    #print(len(np.unique(network_output, axis=0)))


    return (network_input, network_output)

"""
######################################################

LSTM Network Structure

######################################################
"""

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(CuDNNLSTM(512,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
    model.add(Dropout(0.3))
    model.add(Bidirectional(CuDNNLSTM(512, return_sequences=True)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(CuDNNLSTM(512)))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

"""
######################################################

Training

######################################################
"""

def train_network(n_epochs=50):
    """ Train a Neural Network to generate music """

    global STATES_TO_INDEX
    global INDEX_TO_STATES

    # Get list of all states from midi files
    notes = get_notes_and_velocities()

    print(notes.shape)
    print(notes[0])

    # Get the number of unique instances
    # So we can create an embedding!
    n = np.unique(notes, axis=0)
    print(n.shape)

    # The number of unique states/"notes"
    n_vocab = n.shape[0]

    for i, item in enumerate(n):
        STATES_TO_INDEX[str(item.tolist())] = i
        INDEX_TO_STATES[i] = item.tolist()

    print(len(STATES_TO_INDEX))
    print(len(INDEX_TO_STATES))
    print(INDEX_TO_STATES[0]) # match a
    print(INDEX_TO_STATES[1])
    print(str(INDEX_TO_STATES[0])) # match a
    print(STATES_TO_INDEX[str(INDEX_TO_STATES[0])])
    print(STATES_TO_INDEX[str(INDEX_TO_STATES[1])])


    file_path = OUTPUT_FOLDER + '/STATES_TO_INDEX.pkl'
    with open(file_path, "wb") as f:
        pickle.dump(STATES_TO_INDEX, f)

    file_path = OUTPUT_FOLDER + '/INDEX_TO_STATES.pkl'
    with open(file_path, "wb") as f:
        pickle.dump(INDEX_TO_STATES, f)


    # Convert the states into numerical input!
    # Base this on their encodings given unique states
    network_input, network_output = prepare_sequences(notes, n_vocab)

    #print(network_input[0])
    # Set up the model
    model = create_network(network_input, n_vocab)
    history = History()

    

    mc = ModelCheckpoint('./' + OUTPUT_FOLDER + '/LSTMmodel_{epoch:08d}.h5', 
                                     save_weights_only=True, period=20)
    mc_gd = ModelCheckpoint_GenerateData('./' + OUTPUT_FOLDER + '/out_{epoch:08d}', \
        period=50,verbose=True, notes=notes, network_input=network_input)
    try:

        model.summary()
        model.fit(network_input, network_output, epochs=n_epochs, batch_size=64, callbacks=[history,mc, mc_gd], validation_split=0.4)
        model.save('./' + OUTPUT_FOLDER + '/LSTMmodel.h5')

        # Use the model to generate a midi
        prediction_output, random_seed = generate_notes(model, notes, network_input, len(n_vocab))
        #create_midi(prediction_output, 'pokemon_midi')
        file_name = './' + OUTPUT_FOLDER + '/out_' + str(n_epochs)
        create_midi(prediction_output, file_name, notes)

    except KeyboardInterrupt:
        print("Keyboard interrupt")

    # Plot the model losses
    pd.DataFrame(history.history).plot()
    plt.savefig('./' + OUTPUT_FOLDER + '/LSTM_Loss_per_Epoch.png')
    plt.close()



"""
######################################################

Generation of prediction to MIDI

######################################################
"""

def generate_notes(model, notes, network_input, n_vocab, random_seed=None):
    """
        Generate notes from the neural network based on a sequence of notes 

        prediction_output will be a series of states of format [123, 0, 290, ...] and need to be
        converted back to state form with INDEX_TO_STATES


    """
    # pick a random sequence from the input as a starting point for the prediction
    #pitchnames = sorted(set(item for item in notes))
    #PITCHES_MAP

    np.random.seed(random_seed)


    # n = np.unique(notes, axis=0)
    # #print(n[0])
    # print(n.shape)
    # pitchnames = n.tolist()

    start = np.random.randint(0, len(network_input)-1)

    # int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    print("Starting pattern: ")
    print(pattern)
    print(start)
    prediction_output = []

    # generate 2048 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        #print('prediction_input shape:')
        #print(prediction_input.shape)
        #print(prediction_input)
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)
        #print("model's prediction")
        #print(prediction)
        index = np.argmax(prediction)
        #print("max index: ")
        #print(index)
        result = index

        prediction_output.append(result)

        pattern = np.append(pattern,index)
        pattern = pattern[1:len(pattern)]


    return (prediction_output, random_seed)

def create_midi(prediction_output, filename, notes):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []
    print("here in create_midi")
    piano_c_chord = pretty_midi.PrettyMIDI()
    piano = pretty_midi.Instrument(program=0, name='Piano')
    # create note and chord objects based on the values generated by the model
    max_len = len(prediction_output)
    print("max_len")
    print(max_len)
    print(prediction_output)
    on_list = []
    on_list_pos_only = []
    init = 0
    offset = 0

    #print(INDEX_TO_STATES)


    # this signifies a LIST OF STATES
    # where each number in the list represents a list
    for i, state_num in enumerate(prediction_output):
        #print(pattern)
        #print(len(pattern))
        print(state_num)
        pattern = INDEX_TO_STATES[state_num]

        # i = tick, can be converted into time
        # note on [1,1]
        # note sustained [0, 1]
        # note off [0, 0]
        arr_pattern = np.array(pattern)
        #print('pattern')
        #print(arr_pattern)
        #print(arr_pattern.shape)
        indices = np.where(arr_pattern != 0)[0]
        #print(indices)
        indices = indices.tolist()

        #print(indices)
        # check all ones

        for _, pos in enumerate(indices):
            # look at the value of the indices

            # Check for new notes
            # Check if note is on
            if (pos % 3) == 0:
                # in a  position of interval of 3
                if not pos in on_list_pos_only:
                    # this is not a sustain
                    # append [index of state, tick, velocity based on state]
                    on_list.append([pos, i, INDEX_TO_STATES[pos + 2]])
                    on_list_pos_only.append(pos)
            else:
                # odd number position, possibly a velocity or note state
                # check for sustain/consider as new note if it appears
                if init == 1:
                    for j, on in enumerate(on_list):
                        if on[0] == (pos - 1):
                            # for sustain
                            on_list[j][1] = on_list[j][1] + 1 # add another tick to the sustain
                            break
                        elif on[0] == (pos -2):
                            # for velocity
                            on_list[j][2] = 0

            # to always active init after first loop
            init = 1

        # Check if anything from the on_list is now off
        for _, on in enumerate(on_list):
            if (on[0] not in indices):
                # check if not sustain
                if ((on[0] + 1) not in indices):
                    # Remove the note and add to midi file
                    # constant velocity for now
                    start_tick = on[1]
                    end_tick = i
                    #total_ticks = end_tick - start_tick
                    start_time = start_tick * 0.5
                    end_time = end_tick * 0.5
                    print("Appending note: {}".format(INDEX_TO_PITCHES[int(on[0]/3)]))
                    note = pretty_midi.Note(velocity=on[2], pitch=INDEX_TO_PITCHES[int(on[0]/3)], start=start_time, end=end_time)
                    # add it to our piano 
                    piano.notes.append(note)


                    # remove the note
                    on_list_pos_only.remove(on[0])
                    on_list.remove(on)


    # Add the piano instrument to the PrettyMIDI object
    piano_c_chord.instruments.append(piano)
    # Write out the MIDI data
    midi_name = filename + '.mid'
    piano_c_chord.write(midi_name)

    # midi_stream = stream.Stream(output_notes)
    # midi_stream.write('midi', fp='{}.mid'.format(filename))

def generate_given_model_path(file_path, random_seed=None):
    """ Generate a song given the file path to the model 
        and an optional random seed """
    global INDEX_TO_STATES
    global STATES_TO_INDEX

    global PITCHES_TO_INDEX
    global INDEX_TO_PITCHES

    file_path = OUTPUT_FOLDER + '/INDEX_TO_STATES.pkl'
    INDEX_TO_STATES = pickle.load( open( file_path, "rb" ) )

    file_path = OUTPUT_FOLDER + '/STATES_TO_INDEX.pkl'
    STATES_TO_INDEX = pickle.load( open( file_path, "rb" ) )

    file_path = OUTPUT_FOLDER + '/PITCHES_TO_INDEX.pkl'
    PITCHES_TO_INDEX = pickle.load( open( file_path, "rb" ) )

    file_path = OUTPUT_FOLDER + '/INDEX_TO_PITCHES.pkl'
    INDEX_TO_PITCHES = pickle.load( open( file_path, "rb" ) )

    notes = get_notes_and_velocities()


    n = np.unique(notes, axis=0)
    print(n.shape)

    n_vocab = n.shape[0]

    # Convert notes into numerical input
    network_input, network_output = prepare_sequences(notes, n_vocab)


    model = create_network(network_input, n_vocab)
    model.load_weights(file_path)


    # Use the model to generate a midi
    prediction_output, rs = generate_notes(model, notes, network_input, n_vocab, random_seed)
    #create_midi(prediction_output, 'pokemon_midi')
    file_name = './' + OUTPUT_FOLDER + '/out_rand_' + str(rs)
    create_midi(prediction_output, file_name, notes)
    print(file_name)

    return file_name



def continue_training_network(start_epoch, end_epoch):
    global INDEX_TO_STATES
    global STATES_TO_INDEX

    global PITCHES_TO_INDEX
    global INDEX_TO_PITCHES

    file_path = OUTPUT_FOLDER + '/INDEX_TO_STATES.pkl'
    INDEX_TO_STATES = pickle.load( open( "INDEX_TO_STATES.pkl", "rb" ) )

    file_path = OUTPUT_FOLDER + '/STATES_TO_INDEX.pkl'
    STATES_TO_INDEX = pickle.load( open( "STATES_TO_INDEX.pkl", "rb" ) )

    file_path = OUTPUT_FOLDER + '/PITCHES_TO_INDEX.pkl'
    PITCHES_TO_INDEX = pickle.load( open( "PITCHES_TO_INDEX.pkl", "rb" ) )

    file_path = OUTPUT_FOLDER + '/INDEX_TO_PITCHES.pkl'
    INDEX_TO_PITCHES = pickle.load( open( "INDEX_TO_PITCHES.pkl", "rb" ) )

    # Get list of all states from midi files
    notes = get_notes_and_velocities()

    print(notes.shape)
    print(notes[0])

    # Get the number of unique instances
    # So we can create an embedding!
    n = np.unique(notes, axis=0)
    print(n.shape)

    # The number of unique states/"notes"
    n_vocab = n.shape[0]


    print(len(STATES_TO_INDEX))
    print(len(INDEX_TO_STATES))
    print(INDEX_TO_STATES[0]) # match a
    print(INDEX_TO_STATES[1])
    print(str(INDEX_TO_STATES[0])) # match a
    print(STATES_TO_INDEX[str(INDEX_TO_STATES[0])])
    print(STATES_TO_INDEX[str(INDEX_TO_STATES[1])])


    # Convert the states into numerical input!
    # Base this on their encodings given unique states
    network_input, network_output = prepare_sequences(notes, n_vocab)

    #print(network_input[0])
    # Set up the model
    model = create_network(network_input, n_vocab)
    history = History()


    # Load from previous file!
    file_path = OUTPUT_FOLDER + "/LSTMmodel_{:08d}.h5".format(start_epoch)
    model.load_weights(file_path)



    mc = ModelCheckpoint('./' + OUTPUT_FOLDER + '/LSTMmodel_start_' + str(start_epoch) + "_{epoch:08d}.h5", 
                                     save_weights_only=True, period=20)
    mc_gd = ModelCheckpoint_GenerateData('./' + OUTPUT_FOLDER + '/out_{epoch:08d}', \
        period=50,verbose=True, notes=notes, network_input=network_input)

    n_epochs = end_epoch - start_epoch
    try:

        model.summary()
        model.fit(network_input, network_output, epochs=n_epochs, batch_size=64, callbacks=[history,mc, mc_gd], validation_split=0.4)
        model.save('./' + OUTPUT_FOLDER + '/LSTMmodel.h5')

        # Use the model to generate a midi
        prediction_output, random_seed = generate_notes(model, notes, network_input, len(n_vocab))
        #create_midi(prediction_output, 'pokemon_midi')
        file_name = './' + OUTPUT_FOLDER + '/out_' + str(n_epochs)
        create_midi(prediction_output, file_name, notes)

    except KeyboardInterrupt:
        print("Keyboard interrupt")

    # Plot the model losses
    pd.DataFrame(history.history).plot()
    plt.savefig('./' + OUTPUT_FOLDER + '/LSTM_Loss_per_Epoch.png')
    plt.close()


In [ ]:
start_time = time.time()
try:
    continue)
except:
    print("Error in main")

end_time = time.time()
print("Total time elapsed: {}".format(end_time - start_time))



In [5]:

if __name__ == "__main__":
    start_time = time.time()
    try:
        #train_network(500)
        start_epoch = 100
        end_epoch = 500
        continue_training_network(start_epoch, end_epoch)
    except Exception as e:
        print(e)
        print("Error in main")

    end_time = time.time()
    print("Total time elapsed: {}".format(end_time - start_time))



Instrument(program=0, is_drum=False, name="MIDI Out")
1581
27
27
{45: 0, 47: 1, 52: 2, 54: 3, 55: 4, 57: 5, 59: 6, 61: 7, 62: 8, 64: 9, 66: 10, 69: 11, 71: 12, 73: 13, 74: 14, 76: 15, 78: 16, 79: 17, 81: 18, 83: 19, 85: 20, 86: 21, 88: 22, 90: 23, 91: 24, 93: 25, 95: 26}
27
{0: 45, 1: 47, 2: 52, 3: 54, 4: 55, 5: 57, 6: 59, 7: 61, 8: 62, 9: 64, 10: 66, 11: 69, 12: 71, 13: 73, 14: 74, 15: 76, 16: 78, 17: 79, 18: 81, 19: 83, 20: 85, 21: 86, 22: 88, 23: 90, 24: 91, 25: 93, 26: 95}
Parsing ./data/preprocessed/single_piano/midi_bts_boy_with_luv/BTS - Boy With Luv (DJS137).mid
[(0.0, 'note_on', 71, 99), (0.0, 'note_on', 55, 91), (0.0, 'note_on', 59, 91), (0.0, 'note_on', 62, 91), (3.0, 'note_on', 74, 92)]
0
36
0
12
0
18
0
24
3
42
3
36
6
45
6
42
9
48
9
45
12
51
12
48
14
48
14
51
16
18
16
24
16
30
16
12
16
18
16
24
18
48
32
36
32
15
32
21
32
27
32
18
32
24
32
30
35
42
35
36
38
45
38
42
41
39
41
45
44
36
44
39
46
39
46
36
48
15
48
21
48
27
48
15
48
21
48
27
50
39
56
48
56
9
56
15
56
21
56
15
56


48
926
48
926
24
928
45
928
15
928
48
930
21
930
45
932
27
934
33
936
15
938
45
938
21
940
27
940
45
942
45
942
33
944
45
944
0
944
45
946
45
947
6
950
15
952
48
952
9
952
0
952
6
952
15
954
48
955
39
955
18
956
9
957
39
958
24
959
18
960
42
960
12
960
24
963
24
965
42
966
36
968
12
971
24
972
39
974
36
976
42
976
3
976
39
978
9
980
18
980
42
982
24
984
3
986
9
988
48
988
18
990
24
992
45
992
15
992
48
994
21
996
27
996
45
998
33
1000
15
1002
21
1004
27
1006
33
1008
0
1011
6
1014
15
1016
48
1016
9
1016
0
1016
6
1016
15
1018
48
1019
39
1019
18
1020
9
1021
39
1022
42
1022
24
1023
18
1024
57
1024
12
1024
18
1024
24
1024
24
1024
42
1025
57
1025
57
1026
12
1026
18
1026
24
1026
57
1027
12
1027
18
1027
24
1028
54
1029
54
1029
54
1030
54
1032
48
1033
48
1033
48
1034
48
1036
45
1037
12
1037
18
1037
24
1037
45
1038
42
1039
42
1040
57
1040
18
1040
24
1040
30
1041
57
1041
57
1042
18
1042
24
1042
30
1042
57
1043
18
1043
24
1043
30
1044
54
1045
54
1045
54
1046
54
1048
48
1049
48
1049
48
1050
48
1051

48
1614
54
1615
12
1615
18
1615
24
1615
48
1616
18
1616
24
1616
30
1618
54
1618
18
1618
24
1618
30
1619
18
1619
24
1619
30
1619
54
1620
54
1621
18
1621
24
1621
30
1622
18
1622
24
1622
30
1622
54
1622
54
1624
54
1624
18
1624
24
1624
30
1624
54
1625
18
1625
24
1625
30
1626
33
1626
54
1627
18
1627
24
1627
30
1627
33
1628
33
1628
18
1628
24
1628
30
1629
33
1630
48
1631
18
1631
24
1631
30
1631
48
1632
15
1632
21
1632
27
1632
48
1634
15
1634
21
1634
27
1634
48
1635
15
1635
21
1635
27
1636
45
1637
15
1637
21
1637
27
1638
15
1638
21
1638
27
1638
45
1640
45
1640
15
1640
21
1640
27
1641
15
1641
21
1641
27
1643
15
1643
21
1643
27
1643
45
1644
42
1644
15
1644
21
1644
27
1646
45
1646
15
1646
21
1646
27
1646
42
1648
48
1648
15
1648
21
1648
27
1648
45
1650
45
1650
15
1650
21
1650
27
1650
48
1651
15
1651
21
1651
27
1652
45
1652
45
1653
15
1653
21
1653
27
1654
42
1654
15
1654
21
1654
27
1654
45
1656
9
1656
15
1656
21
1656
48
1656
15
1656
21
1656
27
1656
42
1658
9
1658
15
1658
21
1658
48
1659
9
1659
15


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_1 (CuDNNLSTM)     (None, 100, 512)          1054720   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 1024)         4202496   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 1024)         0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1024)              6299648   
_________________________________________________________________
dense_1 (Dense)      

In [17]:
model_path = "/home/agatha/Documents/EE298z/miniproj-2/prelim/music_generation_lstm/output/lstm_single_instrument_v2/midi_bts_boy_with_luv/LSTMmodel_00000100.h5"
start_time = time.time()

generate_given_model_path(model_path, random_seed=1000)

end_time = time.time()
print("Total time elapsed: {}".format(end_time - start_time))



Instrument(program=0, is_drum=False, name="MIDI Out")
1581
27
27
{45: 0, 47: 1, 52: 2, 54: 3, 55: 4, 57: 5, 59: 6, 61: 7, 62: 8, 64: 9, 66: 10, 69: 11, 71: 12, 73: 13, 74: 14, 76: 15, 78: 16, 79: 17, 81: 18, 83: 19, 85: 20, 86: 21, 88: 22, 90: 23, 91: 24, 93: 25, 95: 26}
27
{0: 45, 1: 47, 2: 52, 3: 54, 4: 55, 5: 57, 6: 59, 7: 61, 8: 62, 9: 64, 10: 66, 11: 69, 12: 71, 13: 73, 14: 74, 15: 76, 16: 78, 17: 79, 18: 81, 19: 83, 20: 85, 21: 86, 22: 88, 23: 90, 24: 91, 25: 93, 26: 95}
Parsing ./data/preprocessed/single_piano/midi_bts_boy_with_luv/BTS - Boy With Luv (DJS137).mid
[(0.0, 'note_on', 71, 99), (0.0, 'note_on', 55, 91), (0.0, 'note_on', 59, 91), (0.0, 'note_on', 62, 91), (3.0, 'note_on', 74, 92)]
0
36
0
12
0
18
0
24
3
42
3
36
6
45
6
42
9
48
9
45
12
51
12
48
14
48
14
51
16
18
16
24
16
30
16
12
16
18
16
24
18
48
32
36
32
15
32
21
32
27
32
18
32
24
32
30
35
42
35
36
38
45
38
42
41
39
41
45
44
36
44
39
46
39
46
36
48
15
48
21
48
27
48
15
48
21
48
27
50
39
56
48
56
9
56
15
56
21
56
15
56


748
39
749
39
749
36
750
15
750
21
750
27
752
15
752
21
752
27
753
39
754
15
754
21
754
27
755
15
755
21
755
27
757
15
757
21
757
27
758
36
758
15
758
21
758
27
759
42
759
36
760
9
760
15
760
21
760
48
760
15
760
21
760
27
760
42
762
9
762
15
762
21
763
39
763
9
763
15
763
21
763
48
765
9
765
15
765
21
766
9
766
15
766
21
766
42
766
39
768
12
768
18
768
24
768
9
768
15
768
21
768
42
776
57
778
51
778
57
780
48
780
51
782
45
782
48
784
36
784
18
784
24
784
30
784
12
784
18
784
24
784
45
794
36
800
39
800
15
800
21
800
27
800
18
800
24
800
30
812
39
816
42
816
15
816
21
816
27
816
15
816
21
816
27
820
42
824
30
824
39
824
9
824
15
824
21
824
15
824
21
824
27
828
30
832
12
832
18
832
24
832
9
832
15
832
21
832
39
840
57
842
51
842
57
844
48
844
51
846
45
846
48
848
36
848
18
848
24
848
30
848
12
848
18
848
24
848
45
858
36
860
42
864
39
864
15
864
21
864
27
864
18
864
24
864
30
864
42
874
39
880
42
880
15
880
21
880
27
880
15
880
21
880
27
886
42
888
30
888
39
888
9
888
15
888
21
888
15
8

1464
15
1464
21
1464
60
1464
15
1464
21
1464
27
1468
48
1472
12
1472
18
1472
24
1472
9
1472
15
1472
21
1472
60
1475
12
1475
18
1475
24
1478
12
1478
18
1478
24
1480
57
1481
12
1481
18
1481
24
1482
51
1482
57
1484
48
1484
12
1484
18
1484
24
1484
51
1486
45
1486
48
1487
12
1487
18
1487
24
1488
36
1488
18
1488
24
1488
30
1488
45
1491
18
1491
24
1491
30
1494
18
1494
24
1494
30
1497
18
1497
24
1497
30
1498
36
1500
42
1500
18
1500
24
1500
30
1503
18
1503
24
1503
30
1504
39
1504
15
1504
21
1504
27
1504
42
1507
15
1507
21
1507
27
1510
15
1510
21
1510
27
1513
15
1513
21
1513
27
1514
39
1516
15
1516
21
1516
27
1519
15
1519
21
1519
27
1520
42
1520
15
1520
21
1520
27
1526
42
1528
30
1528
39
1528
9
1528
15
1528
21
1528
15
1528
21
1528
27
1532
30
1532
39
1536
12
1536
18
1536
24
1536
9
1536
15
1536
21
1538
12
1538
18
1538
24
1539
12
1539
18
1539
24
1541
12
1541
18
1541
24
1542
33
1542
54
1542
12
1542
18
1542
24
1543
33
1543
54
1544
33
1544
54
1544
12
1544
18
1544
24
1545
12
1545
18
1545
24
1546
33
154

OSError: Unable to open file (file signature not found)